# train

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')  # build a new model from YAML

# Train the model
model.train(data='./dataset.yaml', epochs=100, imgsz=640, device=0, )

# view sample

In [ ]:
import os
import cv2
import torch
import matplotlib.pyplot as plt
from ultralytics import YOLO
from ultralytics.yolo.engine.results import Results

trained_model = './runs/detect/train7/weights/best.pt'
img_folder_path = '/opt/ml/input/data/det_data/val/images'

model = YOLO(trained_model)

filename = os.listdir(img_folder_path)
i=0

In [ ]:
image_path = os.path.join(img_folder_path, filename[i])
print(image_path)
image = cv2.imread(image_path)
results = model(image)[0]
boxes = results.boxes
class_ = boxes.cls
bounding_boxes = boxes.xyxy

# Matplotlib를 사용하여 이미지와 경계 상자 표시
fig, ax = plt.subplots()
ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

for box, cls in zip(bounding_boxes, class_):
    cls = int(cls.item())
    print("Class:", cls)
    box = [int(item.item()) for item in box]
    x_min, y_min, x_max, y_max = box
    print("Box:", x_min, y_min, x_max, y_max)
    rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                         fill=False, edgecolor='green', linewidth=2)
    ax.add_patch(rect)
    ax.text(x_min, y_min - 5, f'Class: {cls}', color='green')

fig.show()
i+=1

# inference

In [ ]:
import random
from PIL import Image

trained_model = './runs/detect/train7/weights/best.pt'
img_folder_path = '/opt/ml/input/data/det_data/val/images'
train_path_list = trained_model.split('/')
save_root = './output/'+ train_path_list[2] +'/'+ train_path_list[3]

model = YOLO(trained_model)

filenames = os.listdir(img_folder_path)
filenames = random.sample(filenames,100)
for filename in filenames:
    image_path = os.path.join(img_folder_path, filename)
    print(image_path)
    image = Image.open(image_path)
    results = model(image)[0]
    boxes = results.boxes
    class_ = boxes.cls
    bounding_boxes = boxes.xyxy
    for box, cls in zip(bounding_boxes, class_):
        cls = int(cls.item())
        box = [int(item.item()) for item in box]
        x1, y1, x2, y2 = box

        save_path = save_root +'/'+ filename.split('/')[-1][:-4] +'_'+ str(cls) +'.jpg'
        
        cropped_image = image.crop((x1, y1, x2, y2))
        cropped_image.save(save_path)